In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
pwd

'/home/ddkksl123/Model'

In [2]:
# pip install pandas

In [4]:
import pandas as pd
import ast
import warnings
warnings.filterwarnings(action='ignore')

file_path = '/home/ddkksl123/Data/'
file_name = 'data_select.csv'

total_df = pd.read_csv(file_path + file_name) # 02: beaty and sopa
total_df

,V1,V3,V4,V9,V10,V11,V12,V13,V14,V15,...,V23,V24,V25,V26,PCA1,PCA2,PCA3,PCA4,PCA5,Class
0,-2.312227,-1.609851,3.997906,-2.770089,-2.772272,3.202033,-2.899907,-0.595222,-4.289254,0.389724,...,-0.465211,0.320198,0.044519,0.177840,-1.885508,-0.632970,-0.646224,-1.365998,-1.185120,1
1,-3.043541,1.088463,2.288644,-0.270953,-0.838587,-0.414575,-0.503141,0.676502,-1.692029,2.000635,...,1.375966,-0.293803,0.279798,-0.145362,-1.457025,1.361512,-0.733635,-0.628757,1.599017,1
2,-2.303350,-0.359745,2.330243,-0.238253,-1.525412,2.032912,-6.560124,0.022937,-1.470102,-0.698826,...,0.172726,-0.087330,-0.156114,-0.542628,0.640653,-0.429333,0.034827,0.517771,-1.053839,1
3,-4.397974,-2.592844,2.679787,-0.247768,-4.801637,4.895844,-10.912819,0.184372,-6.771097,-0.007326,...,-0.436207,-0.053502,0.252405,-0.657488,-2.845962,-1.092785,0.302198,-0.374536,-0.674921,1
4,1.234235,-4.304597,4.732795,-1.282858,-2.447469,2.101344,-4.609628,1.464378,-6.079337,-0.339237,...,-0.656805,-1.632653,1.488901,0.566797,0.855503,-0.688570,-0.714292,0.609643,-1.889270,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534355,0.120316,-0.546012,-0.745097,-0.204064,-0.657422,0.644837,0.190916,-0.546329,-0.731707,-0.808036,...,0.050343,0.102800,-0.435870,0.124079,0.606253,-0.107399,-0.458092,0.201008,-0.537071,0
534356,-11.881118,-9.834783,-2.066656,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,...,1.014480,-0.509348,1.436807,0.250034,-2.969463,0.094410,-4.636823,-4.865279,-5.425135,0
534357,-0.732789,2.035030,-0.738589,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,...,0.012463,-1.016226,-0.606624,-0.395255,0.270690,0.610748,0.356408,-0.360807,0.047825,0
534358,1.919565,-3.249640,-0.557828,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,...,-0.037501,0.640134,0.265745,-0.087371,0.703074,1.598169,0.872046,-0.595462,0.306705,0


In [5]:
!pip install pycaret
!pip install Jinja2
!pip install markupsafe==2.1.1
!pip install xgboost
!pip install catboost

In [8]:
pip install --upgrade Jinja2

     |████████████████████████████████| 133 kB 8.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
total_df

train = total_df.drop(columns = 'Class', axis=1)
target = total_df['Class'].values

In [11]:
from pycaret.classification import *
import pandas as pd

# 데이터 샘플링
sampled_data = total_df.sample(frac=0.2, random_state=123)  # 예를 들어, 데이터의 50%만 사용
sampled_train = sampled_data.drop(columns='Class', axis=1)
sampled_target = sampled_data['Class']

# PyCaret 설정
setup_clf = setup(data=sampled_train, target=sampled_target, train_size=0.7,
                  fold=5, fold_shuffle=True,
                  feature_selection=True, feature_selection_method='classic',
                  use_gpu=True, session_id=123)

# 모델 비교
top5 = compare_models(n_select=5, fold=5)

# 튜닝할 파라미터 제한 (튜닝 시간 단축)
tuned_top5 = [tune_model(i, optimize='Accuracy', n_iter=10, choose_better=True) for i in top5]

# 앙상블
blender_top5 = blend_models(estimator_list=tuned_top5)
final_model = finalize_model(blender_top5)

# 예측
prediction = predict_model(final_model, data=total_df.iloc[-100:])

# 모델 저장
save_model(final_model, file_path + 'pycaret_sample')


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the prov

,Description,Value
0,Session id,123
1,Target,Class
2,Target type,Binary
3,Original data shape,"(106872, 25)"
4,Transformed data shape,"(106872, 5)"
5,Transformed train set shape,"(74810, 5)"
6,Transformed test set shape,"(32062, 5)"
7,Numeric features,24
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the prov

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9888,0.9996,0.9964,0.9800,0.9881,0.9775,0.9776,1.3260
rf,Random Forest Classifier,0.9849,0.9989,0.9947,0.9737,0.9841,0.9697,0.9699,3.0100
dt,Decision Tree Classifier,0.9730,0.9733,0.9789,0.9641,0.9715,0.9458,0.9459,0.8800
knn,K Neighbors Classifier,0.9716,0.9900,0.9983,0.9443,0.9706,0.9431,0.9445,1.0400
xgboost,Extreme Gradient Boosting,0.9671,0.9945,0.9837,0.9481,0.9656,0.9340,0.9346,0.8180
catboost,CatBoost Classifier,0.9523,0.9901,0.9711,0.9303,0.9503,0.9045,0.9052,4.4260
lightgbm,Light Gradient Boosting Machine,0.9507,0.9901,0.9717,0.9268,0.9487,0.9013,0.9023,0.7680
gbc,Gradient Boosting Classifier,0.9035,0.9676,0.9082,0.8887,0.8983,0.8065,0.8067,10.6720
ada,Ada Boost Classifier,0.8594,0.9426,0.8292,0.8656,0.8470,0.7170,0.7176,3.0420
lr,Logistic Regression,0.8373,0.9179,0.7858,0.8558,0.8193,0.6719,0.6739,0.6640


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8447,0.9363,0.7970,0.8619,0.8282,0.6870,0.6887
1,0.8451,0.9355,0.8034,0.8576,0.8296,0.6879,0.6891
2,0.8457,0.9311,0.8085,0.8550,0.8311,0.6893,0.6903
3,0.8498,0.9378,0.8098,0.8618,0.8350,0.6973,0.6985
4,0.8455,0.9361,0.8005,0.8606,0.8294,0.6885,0.6900
Mean,0.8462,0.9354,0.8038,0.8594,0.8307,0.6900,0.6913
Std,0.0018,0.0022,0.0048,0.0027,0.0023,0.0037,0.0036


...

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Info] Number of positive: 28093, number of negative: 31755
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003564 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6120
[LightGBM] [Info] Number of data points in the train set: 59848, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.469406 -> initscore=-0.122530
[LightGBM] [Info] Start training from score -0.122530
[LightGBM] [Info] Number of positive: 28093, number of negative: 31755
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003559 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6120
[LightGBM] [Info] Number of data points in the train set: 59848, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.469406 -> initscore=-0.122530
[Ligh

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9836,0.9988,0.9967,0.9693,0.9828,0.9672,0.9675
1,0.9852,0.9993,0.9972,0.9721,0.9845,0.9704,0.9707
2,0.9818,0.9988,0.9966,0.9658,0.9809,0.9636,0.9640
3,0.9836,0.9990,0.9957,0.9700,0.9827,0.9670,0.9673
4,0.9839,0.9991,0.9962,0.9703,0.9831,0.9677,0.9680
Mean,0.9836,0.9990,0.9965,0.9695,0.9828,0.9672,0.9675
Std,0.0011,0.0002,0.0005,0.0021,0.0011,0.0022,0.0021


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9900,0,0.0000,0.0000,0.0000,0.0000,0.0000


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['V1', 'V3', 'V4', 'V9', 'V10',
                                              'V11', 'V12', 'V13', 'V14', 'V15',
                                              'V16', 'V17', 'V18', 'V19', 'V22',
                                              'V23', 'V24', 'V25', 'V26', 'PCA1',
                                              'PCA2', 'PCA3', 'PCA4', 'PCA5'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False...
                                                              max_cat_threshold=None,
                                                      